In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [3]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FreeThrowsMade&agg=PerGame"
years = list(range(2001, 2022))

In [5]:
for year in years:
    with open("free_throws_made_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [7]:
dfs = []
for year in years:
    with open("free_throws_made_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    freethrows_made = pd.read_html(str(player_table))[0]
    freethrows_made["Year"] = year
    dfs.append(freethrows_made)

In [8]:
freethrows_made= pd.concat(dfs)
freethrows_made

,Rank,Player,Team,GamesG,AttemptedAtt.,Made,% Made%,Year
0,1,"LAKOVIC, JAKALAKOVIC, J.",MESKRKA Novo MestoKRKA Novo Mesto,14,7.14,6.36,89.00%,2001
1,2,"SKELIN, MATESKELIN, M.",MESKRKA Novo MestoKRKA Novo Mesto,14,8.57,6.00,70.00%,2001
2,3,"BODIROGA, DEJANBODIROGA, D.",PAOPanathinaikos AthensPanathinaikos Athens,22,7.36,5.86,79.63%,2001
3,4,"LUKOVSKI, DRAGANLUKOVSKI, D.",PAUPau-OrthezPau-Orthez,14,6.36,5.43,85.39%,2001
4,5,"ALEXIS, WENDELLALEXIS, W.",BERAlba BerlinAlba Berlin,14,6.29,5.36,85.23%,2001
...,...,...,...,...,...,...,...,...
45,46,"POIRIER, VINCENTPOIRIER, V.",RMBReal MadridReal Madrid,37,2.22,1.70,76.83%,2021
46,47,"DUNSTON, BRYANTDUNSTON, B.",EFSAnadolu Efes IstanbulAnadolu Efes Istanbul,37,2.41,1.68,69.66%,2021
47,48,"CANAAN, ISAIAHCANAAN, I.",UNKUNICS KazanUNICS Kazan,24,2.04,1.67,81.63%,2021
48,49,"POYTHRESS, ALEXPOYTHRESS, A.",ZENZenit St PetersburgZenit St Petersburg,23,2.04,1.57,76.60%,2021


In [9]:
engine = create_engine('sqlite:///freethrows_made_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [10]:
freethrows_made.to_sql(
    name = "freethrows_maded_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [11]:
freethrows_maded_leaders_select = session.execute('select * from "freethrows_maded_leaders"')
freethrows_maded_leaders_select.all()

[(0, 1, 'LAKOVIC, JAKALAKOVIC, J.', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 7.14, 6.36, '89.00%', 2001),
 (1, 2, 'SKELIN, MATESKELIN, M.', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 8.57, 6.0, '70.00%', 2001),
 (2, 3, 'BODIROGA, DEJANBODIROGA, D.', 'PAOPanathinaikos AthensPanathinaikos Athens', 22, 7.36, 5.86, '79.63%', 2001),
 (3, 4, 'LUKOVSKI, DRAGANLUKOVSKI, D.', 'PAUPau-OrthezPau-Orthez', 14, 6.36, 5.43, '85.39%', 2001),
 (4, 5, 'ALEXIS, WENDELLALEXIS, W.', 'BERAlba BerlinAlba Berlin', 14, 6.29, 5.36, '85.23%', 2001),
 (5, 6, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 6.75, 5.35, '79.26%', 2001),
 (6, 7, 'AVLEEV, RUSLANAVLEEV, R.', 'PEMUral GreatUral Great', 20, 6.5, 5.3, '81.54%', 2001),
 (7, 8, 'SCEPANOVIC, VLADOSCEPANOVIC, V.', 'PARPartizanPartizan', 14, 6.29, 5.07, '80.68%', 2001),
 (8, 8, 'KHIZHNIAK, GRYGORIIKHIZHNIAK, G.', 'ZALZalgirisZalgiris', 14, 5.71, 5.07, '88.75%', 2001),
 (9, 10, 'BROWN, MARCUSBROWN, M.', 'EFSEfes PilsenEfes Pilsen', 19, 5.68, 5.0, '87.96%